<a href="https://colab.research.google.com/github/hamzzihyun/competition/blob/main/DACON/%EC%9D%8C%EC%95%85%EC%9E%A5%EB%A5%B4%EB%B6%84%EB%A5%98AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 설치 시작
!pip uninstall sklearn -y
!pip install --upgrade sklearn
!pip install scikit-learn==0.23.2 --user

import sklearn

!pip install pycaret
# 실행 후 런타임 재시작

In [ ]:
# 라이브러리 설치
!pip install catboost

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/DACON/basic/음악 장르 분류 AI/open.zip') as z:
  z.extractall()

## Import

In [ ]:
import sys
import random
import pandas as pd
import numpy as np
import os
import torch
import pickle
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

from tqdm import tqdm
import sklearn
import warnings
import catboost
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import torch
use_cuda = torch.cuda.is_available()
print(use_cuda)

True


## Fixed RandomSeed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## Data Load

In [ ]:
train = pd.read_csv("./train.csv").drop(columns=['ID'])
test = pd.read_csv("./test.csv").drop(columns=['ID'])

In [ ]:
print(train.shape)
print(train.columns)
train.head()

(25383, 12)
Index(['danceability', 'energy', 'key', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration', 'genre'],
      dtype='object')


,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration,genre
0,0.3341,0.9231,10,0.8540,0.3797,0.0080,0.0002,0.7697,0.7902,0.5391,175,trap
1,0.6344,0.5955,10,0.2826,0.1378,0.0000,0.7137,0.1597,0.4162,0.3043,513,techno
2,0.3983,0.1107,8,0.2459,0.0247,0.7740,0.0000,0.5105,0.4291,0.2324,151,Trap Metal
3,0.3139,0.9702,2,0.9252,0.9590,0.0371,0.3015,0.2539,0.0577,0.5406,246,trap
4,0.0735,0.9452,5,0.7702,0.1397,0.2583,0.0015,0.1774,0.1128,0.3176,229,Dark Trap


In [ ]:
print(test.shape)
print(test.columns)
test.head()

(16922, 11)
Index(['danceability', 'energy', 'key', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration'],
      dtype='object')


,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration
0,0.8861,0.6821,2,0.7241,0.3103,0.1451,0.0000,0.5087,0.8863,0.9648,279
1,0.8495,0.3883,1,0.4495,0.4661,0.6234,0.0004,0.1347,0.4062,0.9051,151
2,0.2479,0.7549,3,0.7683,0.3131,0.5718,0.0000,0.1250,0.6357,0.3234,196
3,0.6344,0.9804,1,0.6599,0.0931,0.0020,0.6016,0.8799,0.8790,0.2946,207
4,0.6237,0.6926,6,0.6716,0.0269,0.0283,0.0001,0.1580,0.9288,0.3642,196


## Modeling

In [ ]:
# model comparison
from pycaret.classification import *
my_model = setup(session_id = 42,
                data = train,
                train_size = 0.7, 
                target ='genre',
                numeric_features=list(train.drop(columns=['genre']).columns),
                normalize = True,
                fold_shuffle = True,
                fold = 10,
                use_gpu = True) 

,Description,Value
0,session_id,42
1,Target,genre
2,Target Type,Multiclass
3,Label Encoded,"Dark Trap: 0, Emo: 1, Hiphop: 2, Pop: 3, Rap: ..."
4,Original Data,"(25383, 12)"
5,Missing Values,False
6,Numeric Features,11
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=['danceability',
                                                          'energy', 'key',
                                                          'loudness',
                                                          'speechiness',
                                                          'acousticness',
                                                          'instrumentalness',
                                                          'liveness', 'valence',
                                                    

In [ ]:
# 모델 비교
model = compare_models(sort='F1', fold=3, n_select=3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.6719,0.9482,0.6275,0.6706,0.6647,0.6422,0.6437,42.0733
catboost,CatBoost Classifier,0.6683,0.9479,0.6292,0.6662,0.6637,0.6387,0.6396,24.4567
lightgbm,Light Gradient Boosting Machine,0.6665,0.9464,0.6250,0.6660,0.6624,0.6364,0.6375,2.7700
rf,Random Forest Classifier,0.6610,0.9370,0.6137,0.6586,0.6534,0.6301,0.6315,2.4600
et,Extra Trees Classifier,0.6533,0.9119,0.6051,0.6464,0.6431,0.6216,0.6233,2.8633
qda,Quadratic Discriminant Analysis,0.6112,0.9357,0.5942,0.6282,0.6100,0.5789,0.5806,0.1033
nb,Naive Bayes,0.5855,0.9278,0.5739,0.6011,0.5843,0.5517,0.5534,0.0667
lr,Logistic Regression,0.5832,0.9244,0.5302,0.5780,0.5714,0.5453,0.5468,1.3733
dt,Decision Tree Classifier,0.5477,0.7568,0.5219,0.5464,0.5467,0.5095,0.5096,0.1933
knn,K Neighbors Classifier,0.5472,0.8534,0.5111,0.5365,0.5391,0.5077,0.5082,1.0967


INFO:logs:create_model_container: 30
INFO:logs:master_model_container: 30
INFO:logs:display_container: 3
INFO:logs:[GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False), <catboost.core.CatBoostClassifier object at 0x7f0e83c655e0>, LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_

In [ ]:
# 모델 튜닝
tuned_model = [tune_model(i) for i in model]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6877,0.9519,0.6560,0.6863,0.6819,0.6599,0.6611
1,0.6888,0.9514,0.6516,0.6813,0.6805,0.6610,0.6622
2,0.6787,0.9525,0.6345,0.6753,0.6727,0.6498,0.6511
3,0.6832,0.9522,0.6419,0.6830,0.6786,0.6546,0.6559
4,0.6832,0.9545,0.6385,0.6836,0.6793,0.6548,0.6558
5,0.6792,0.9511,0.6448,0.6764,0.6736,0.6509,0.6516
6,0.6646,0.9471,0.6146,0.6626,0.6581,0.6343,0.6356
7,0.6618,0.9460,0.6294,0.6631,0.6557,0.6314,0.6334
8,0.6830,0.9487,0.6394,0.6812,0.6751,0.6541,0.6560


INFO:logs:create_model_container: 33
INFO:logs:master_model_container: 33
INFO:logs:display_container: 6
INFO:logs:LGBMClassifier(bagging_fraction=0.6, bagging_freq=5, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', learning_rate=0.05, max_depth=-1,
               min_child_samples=86, min_child_weight=0.001, min_split_gain=0.9,
               n_estimators=130, n_jobs=-1, num_leaves=40, objective=None,
               random_state=42, reg_alpha=2, reg_lambda=0.001, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
INFO:logs:tune_model() succesfully completed......................................


In [ ]:
tuned_model

[GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.05, loss='deviance', max_depth=6,
                            max_features='sqrt', max_leaf_nodes=None,
                            min_impurity_decrease=0.0002,
                            min_impurity_split=None, min_samples_leaf=3,
                            min_samples_split=5, min_weight_fraction_leaf=0.0,
                            n_estimators=120, n_iter_no_change=None,
                            presort='deprecated', random_state=42,
                            subsample=0.65, tol=0.0001, validation_fraction=0.1,
                            verbose=0, warm_start=False),
 LGBMClassifier(bagging_fraction=0.6, bagging_freq=5, boosting_type='gbdt',
                class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
                importance_type='split', learning_rate=0.05, max_depth=-1,
                min_child_samples=86, min_child_weight=0.

In [ ]:
# 앙상블
blended = blend_models(estimator_list = tuned_model,
                       method = 'soft',
                       optimize='F1',
                       )

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6956,0.9539,0.6579,0.6945,0.6894,0.6682,0.6698
1,0.6927,0.9530,0.6507,0.6923,0.6846,0.6649,0.6667
2,0.6911,0.9544,0.6455,0.6928,0.6847,0.6629,0.6650
3,0.6927,0.9537,0.6437,0.6940,0.6871,0.6648,0.6663
4,0.6849,0.9552,0.6377,0.6892,0.6791,0.6562,0.6577
5,0.6764,0.9517,0.6376,0.6782,0.6700,0.6473,0.6485
6,0.6685,0.9479,0.6176,0.6681,0.6603,0.6383,0.6401
7,0.6685,0.9474,0.6325,0.6718,0.6616,0.6385,0.6408
8,0.6819,0.9514,0.6396,0.6837,0.6734,0.6528,0.6548


INFO:logs:create_model_container: 34
INFO:logs:master_model_container: 34
INFO:logs:display_container: 7
INFO:logs:VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.05,
                                                         loss='deviance',
                                                         max_depth=6,
                                                         max_features='sqrt',
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0002,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=3,
  

In [ ]:
# 모델 성능평가
final_model = finalize_model(blended)
evaluate_model(final_model)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.05,
                                                         loss='deviance',
                                                         max_depth=6,
                                                         max_features='sqrt',
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0002,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=3,
                                          

[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 

INFO:logs:Calculating mean and std
INFO:logs:Creating metrics dataframe
INFO:logs:Finalizing model
INFO:logs:create_model_container: 34
INFO:logs:master_model_container: 34
INFO:logs:display_container: 8
INFO:logs:VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.05,
                                                         loss='deviance',
                                                         max_depth=6,
                                                         max_features='sqrt',
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0002,
                                                         min_i

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

## Submit

In [ ]:
prediction = predict_model(final_model, data = test)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.05,
                                                         loss='deviance',
                                                         max_depth=6,
                                                         max_features='sqrt',
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0002,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=3,
                                            

In [ ]:
# 제출 파일을 불러옵니다.
submission = pd.read_csv("./sample_submission.csv")
submission["genre"] = prediction['Label']

In [ ]:
submission

,ID,genre
0,TEST_00000,Hiphop
1,TEST_00001,Underground Rap
2,TEST_00002,Emo
3,TEST_00003,techhouse
4,TEST_00004,Pop
...,...,...
16917,TEST_16917,techhouse
16918,TEST_16918,Hiphop
16919,TEST_16919,hardstyle
16920,TEST_16920,psytrance


In [ ]:
submission.to_csv("./submit(7).csv", index = False)